In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )


intitate_notebook()

Current directory for notebook:  /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project


In [2]:
from pyspark.sql import SparkSession
from notebooks.utils import (
    column_stats,
    get_column_description,
    get_columns_high_missing_values,
)

In [3]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_person_1.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_0").getOrCreate()

df_person_1 = spark.read.parquet(str(parquet_file_path))
df_person_1.show()

24/07/17 11:40:46 WARN Utils: Your hostname, Shirshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.109 instead (on interface en0)
24/07/17 11:40:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/17 11:40:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/17 11:40:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/17 11:40:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------+-------------+-------------+---------------------+------------------------+------------------------+---------------------+--------------+----------------------+-----------------------+------------------+----------------------+---------------------+----------------+-----------+--------------+----------------+----------------+----------------+--------------+----------------------+--------------+----------+-----------------+----------------+---------------+-------------------------+-----------------------+-------------------------+-------------------------+-------------+----------+---------+---------------------+--------+--------------+
|case_id|birth_259D|birthdate_87D|childnum_185L|contaddr_district_15M|contaddr_matchlist_1032L|contaddr_smempladdr_334L|contaddr_zipcode_807M|education_927M|empl_employedfrom_271D|empl_employedtotal_800L|empl_industry_691L|empladdr_district_926M|empladdr_zipcode_114M|familystate_447L|gender_992L|housetype_905L|housingtype_772L|incometyp

In [4]:
df_person_1 = df_person_1.filter(df_person_1.num_group1 == 0)
df_person_1.show()

+-------+----------+-------------+-------------+---------------------+------------------------+------------------------+---------------------+--------------+----------------------+-----------------------+------------------+----------------------+---------------------+----------------+-----------+--------------+----------------+--------------------+----------------+--------------+----------------------+--------------+----------+-----------------+----------------+---------------+-------------------------+-----------------------+-------------------------+-------------------------+-------------+----------+---------+---------------------+--------+--------------+
|case_id|birth_259D|birthdate_87D|childnum_185L|contaddr_district_15M|contaddr_matchlist_1032L|contaddr_smempladdr_334L|contaddr_zipcode_807M|education_927M|empl_employedfrom_271D|empl_employedtotal_800L|empl_industry_691L|empladdr_district_926M|empladdr_zipcode_114M|familystate_447L|gender_992L|housetype_905L|housingtype_772L|    i

In [6]:
missing_values_dict = get_columns_high_missing_values(
    df=df_person_1, threshold=20, display=True
)

100%|██████████| 37/37 [00:05<00:00,  6.73it/s]


In [12]:
df_person_1 = df_person_1.drop(*list(missing_values_dict.keys())+["num_group1"])
df_person_1.show()

+-------+----------+---------------------+------------------------+------------------------+---------------------+--------------+----------------------+---------------------+--------------------+--------------+----------------------+-----------------+----------------+---------------+-------------------------+-----------------------+----------+---------------------+--------+--------------+
|case_id|birth_259D|contaddr_district_15M|contaddr_matchlist_1032L|contaddr_smempladdr_334L|contaddr_zipcode_807M|education_927M|empladdr_district_926M|empladdr_zipcode_114M|    incometype_1044T|language1_981M|mainoccupationinc_384A|personindex_1023L|persontype_1072L|persontype_792L|registaddr_district_1083M|registaddr_zipcode_184M|role_1084L|safeguarantyflag_411L|sex_738L|      type_25L|
+-------+----------+---------------------+------------------------+------------------------+---------------------+--------------+----------------------+---------------------+--------------------+--------------+------

24/07/17 14:36:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1068357 ms exceeds timeout 120000 ms
24/07/17 14:36:44 WARN SparkContext: Killing executors is not supported by current scheduler.
24/07/17 14:36:44 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

##### Noted: 
I think Person Details are Depth=2 level is not required. Depth=1 Level is eoungh.

In [9]:
# column_stats(df_person_1, list(missing_values_dict.keys())[-1])
# df_person_1.select("familystate_447L").distinct().show()

In [10]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_person_2.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_0").getOrCreate()

df_person_2 = spark.read.parquet(str(parquet_file_path))
df_person_2.show()

+-------+--------------------+----------------+---------------+--------------+-----------------------+-----------------------+------------------------+----------+----------+------------------------+
|case_id|addres_district_368M|addres_role_871L|addres_zip_823M|conts_role_79M|empls_economicalst_849M|empls_employedfrom_796D|empls_employer_name_740M|num_group1|num_group2|relatedpersons_role_762T|
+-------+--------------------+----------------+---------------+--------------+-----------------------+-----------------------+------------------------+----------+----------+------------------------+
|      5|            a55475b1|            NULL|       a55475b1|      a55475b1|               a55475b1|                   NULL|                a55475b1|         0|         0|                    NULL|
|      6|          P55_110_32|         CONTACT|      P10_68_40|    P38_92_157|            P164_110_33|                   NULL|                a55475b1|         0|         0|                    NULL|
|    

In [11]:
column_stats(df_person_2, "case_id")

Column Name: case_id
Minimum Value: 5
Maximum Value: 2703454
Missing Values Count: 0 (0.00%)
Number of Unique Values: 1435105
Total Number of Rows: 1643410
Column Name:  case_id
Description - Not Found
---------------------------------------------------------------------------------------------------- 




'Function - column_stats - Done'

In [ ]:
# for column_name in df_person_2.columns:
#     column_stats(df_person_2, column_name)